In [96]:
#import libraries
import os
import sys
import json
import numpy as np
import urllib2
import urllib
import time

#HDFS cli data fetching
from hdfs import Config
import pandas as pd
import Queue
from hdfs.ext.avro import AvroReader, AvroWriter
from hdfs.ext.dataframe import read_dataframe, write_dataframe

#need to set default encoding or foreign languages wont work
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

client = Config().get_client('holdem') #Get a client

In [97]:
job_translations = {
"de_DE": "stellen",
"ru_RU": "jobs",
"it_IT": "offerte-di-lavoro",
"pl_PL": "jobs",
"sv_SE": "jobs",
"da_DK": "jobs",
"tr_TR": "jobs",
"pt_BR": "jobs",
"ja_JP": "jobs",
"fr_FR": "emplois",
"cs_CZ": "jobs",
"fi_FI": "jobs",
"ko_KR": "jobs",
"es_ES": "empleos",
"zh_CN": "职位",
"no_NO": "jobs",
"ar_AE": "jobs",
"nl_NL": "jobs",
"en_US": "jobs"
}

In [248]:
country_mappings = {
"de_DE": [2276],
"ru_RU": [2643],
"it_IT": [2380],
"pl_PL": [2616],
"sv_SE": [2752],
"da_DK": [2208],
"tr_TR": [2792],
#"pt_BR": [2076, 2620], #brazil and portugal
"pt_BR": [2076], #brazil only
"ja_JP": [2392],
"fr_FR": [2250],
"cs_CZ": [2203],
"fi_FI": [2246],
"ko_KR": [2410],
"es_ES": [2724],
"zh_CN": [2156],
"no_NO": [2578],
"ar_AE": [2784],
"nl_NL": [2528],
#"en_US": [2840,2826,2036,2702,2356,2124]
"en_US": [2840] #us only
}

In [165]:
schema = {
      'doc': 'Google Ads Volume Metrics',
      'name': 'googleads',
      'fields': [
        {'name': 'id', 'type': 'string'},
        {'name': 'search_keyword', 'type': 'string'}, 
        {'name': 'volume', 'type': 'string'},
        {'name': 'location', 'type': 'string'},
        {'name': 'keyword', 'type': 'string'}
      ],
      'type': 'record'
}

In [166]:
pandas_schema = ['id'
                 ,'search_keyword'
                 ,'volume'
                 ,'location'
                 ,'keyword']

In [6]:
#read in the file of keywords
scoreFile = '/data/derived/standardization/taxonomy/title/5/5.1.0/titles.avro'  #File path on HDFS
df3 = read_dataframe(client, scoreFile)
df3.head(10)

,acronyms,aliases,blacklistStatus,dimensions,displayNames,functions,granularity,id,isOccupation,isRole,machineTranslatedName,misspellings,names,preferredTitle,seniority,status,superTitles,vague,wikipediaUrl
0,{},{u'en_US': [u'consultant sap technology']},ALLOWED,{u'roleId': 82},{},[],TITLE,12005,True,False,"{u'ru_RU': u'сок техническии консультант', u'r...",{},"{u'ru_RU': u'Консультант по SAP-технологиям', ...",NaN,4.0,ACTIVE,[urn:li:superTitle:441],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
1,{},{},ALLOWED,{u'roleId': 2},{},[],TITLE,21391,True,False,"{u'ru_RU': u'диетическое менеджер', u'ro_RO': ...",{},"{u'ru_RU': u'Менеджер-диетолог', u'ar_AE': u'م...",NaN,5.0,ACTIVE,[urn:li:superTitle:317],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
2,{},{u'en_US': [u'branch manager office']},ALLOWED,None,{},[],TITLE,9380,False,False,"{u'ro_RO': u'manager de sucursala', u'fr_FR': ...",{},{u'en_US': u'Branch Office Manager'},NaN,4.0,ACTIVE,[urn:li:superTitle:223],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
3,{},{u'en_US': [u'manager services shared']},ALLOWED,None,{},[],TITLE,6914,False,False,{u'pt_BR': u'gerente de servicos compartilhado...,{},{u'ru_RU': u'Менеджер по общекорпоративным сер...,NaN,5.0,ACTIVE,[urn:li:superTitle:245],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
4,{},{u'en_US': [u'business development operations ...,ALLOWED,{u'roleId': 114},{},[],TITLE,10484,True,False,{u'ru_RU': u'вице президент операции по развит...,{},{u'ru_RU': u'Вице-президент по управлению и ра...,NaN,7.0,ACTIVE,[urn:li:superTitle:469],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
5,{},{u'ru_RU': [u'руководитель отдела разработки с...,ALLOWED,None,{},[],TITLE,5139,False,True,"{u'pt_BR': u'gerente de frequencia de radio', ...",{},"{u'ru_RU': u'Диспетчер радиосвязи', u'ar_AE': ...",NaN,5.0,ACTIVE,[urn:li:superTitle:333],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
6,{},"{u'ru_RU': [u'Командующий', u'начальник'], u'f...",ALLOWED,None,{},[],TITLE,8904,False,True,"{u'ru_RU': u'Командующий', u'da_DK': u'kommand...",{},"{u'ru_RU': u'Комендант', u'ar_AE': u'آمر', u'f...",NaN,5.0,ACTIVE,[urn:li:superTitle:442],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
7,{},{u'en_US': [u'hardware manager project']},ALLOWED,{u'roleId': 4},{},[],TITLE,21327,True,False,"{u'ru_RU': u'менеджер аппаратных проект', u'ro...",{},{u'ru_RU': u'Руководитель проекта по установке...,NaN,4.0,ACTIVE,[urn:li:superTitle:483],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
8,{},{},ALLOWED,{u'roleId': 167},{},[],TITLE,11928,True,False,"{u'ru_RU': u'сердечно сосудистои специалист', ...",{},{u'ru_RU': u'Специалист в области сердечнососу...,NaN,3.0,ACTIVE,[urn:li:superTitle:620],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
9,{},{u'en_US': [u'advanced application business co...,ALLOWED,None,{},[],TITLE,14729,False,False,{u'ru_RU': u'старшии сок консультант расширенн...,{},{u'en_US': u'Senior Sap Advanced Business Appl...,NaN,4.0,ACTIVE,[urn:li:superTitle:441],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None


In [273]:
df3.head(10)

,acronyms,aliases,blacklistStatus,dimensions,displayNames,functions,granularity,id,isOccupation,isRole,machineTranslatedName,misspellings,names,preferredTitle,seniority,status,superTitles,vague,wikipediaUrl
0,{},{u'en_US': [u'consultant sap technology']},ALLOWED,{u'roleId': 82},{},[],TITLE,12005,True,False,"{u'ru_RU': u'сок техническии консультант', u'r...",{},"{u'ru_RU': u'Консультант по SAP-технологиям', ...",NaN,4.0,ACTIVE,[urn:li:superTitle:441],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
1,{},{},ALLOWED,{u'roleId': 2},{},[],TITLE,21391,True,False,"{u'ru_RU': u'диетическое менеджер', u'ro_RO': ...",{},"{u'ru_RU': u'Менеджер-диетолог', u'ar_AE': u'م...",NaN,5.0,ACTIVE,[urn:li:superTitle:317],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
2,{},{u'en_US': [u'branch manager office']},ALLOWED,None,{},[],TITLE,9380,False,False,"{u'ro_RO': u'manager de sucursala', u'fr_FR': ...",{},{u'en_US': u'Branch Office Manager'},NaN,4.0,ACTIVE,[urn:li:superTitle:223],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
3,{},{u'en_US': [u'manager services shared']},ALLOWED,None,{},[],TITLE,6914,False,False,{u'pt_BR': u'gerente de servicos compartilhado...,{},{u'ru_RU': u'Менеджер по общекорпоративным сер...,NaN,5.0,ACTIVE,[urn:li:superTitle:245],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
4,{},{u'en_US': [u'business development operations ...,ALLOWED,{u'roleId': 114},{},[],TITLE,10484,True,False,{u'ru_RU': u'вице президент операции по развит...,{},{u'ru_RU': u'Вице-президент по управлению и ра...,NaN,7.0,ACTIVE,[urn:li:superTitle:469],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
5,{},{u'ru_RU': [u'руководитель отдела разработки с...,ALLOWED,None,{},[],TITLE,5139,False,True,"{u'pt_BR': u'gerente de frequencia de radio', ...",{},"{u'ru_RU': u'Диспетчер радиосвязи', u'ar_AE': ...",NaN,5.0,ACTIVE,[urn:li:superTitle:333],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
6,{},"{u'ru_RU': [u'Командующий', u'начальник'], u'f...",ALLOWED,None,{},[],TITLE,8904,False,True,"{u'ru_RU': u'Командующий', u'da_DK': u'kommand...",{},"{u'ru_RU': u'Комендант', u'ar_AE': u'آمر', u'f...",NaN,5.0,ACTIVE,[urn:li:superTitle:442],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
7,{},{u'en_US': [u'hardware manager project']},ALLOWED,{u'roleId': 4},{},[],TITLE,21327,True,False,"{u'ru_RU': u'менеджер аппаратных проект', u'ro...",{},{u'ru_RU': u'Руководитель проекта по установке...,NaN,4.0,ACTIVE,[urn:li:superTitle:483],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
8,{},{},ALLOWED,{u'roleId': 167},{},[],TITLE,11928,True,False,"{u'ru_RU': u'сердечно сосудистои специалист', ...",{},{u'ru_RU': u'Специалист в области сердечнососу...,NaN,3.0,ACTIVE,[urn:li:superTitle:620],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None
9,{},{u'en_US': [u'advanced application business co...,ALLOWED,None,{},[],TITLE,14729,False,False,{u'ru_RU': u'старшии сок консультант расширенн...,{},{u'en_US': u'Senior Sap Advanced Business Appl...,NaN,4.0,ACTIVE,[urn:li:superTitle:441],"{u'isVerified': True, u'vaguenessLevel': 0.0}",None


In [37]:
len(df3)

24701

In [300]:
keyword_dict = df3.iloc[3]['names']
keyword_dict

{u'ar_AE': u'\u0645\u062f\u064a\u0631 \u062e\u062f\u0645\u0627\u062a \u0645\u0634\u062a\u0631\u0643\u0629',
 u'cs_CZ': u'Mana\u017eer sd\xedlen\xfdch slu\u017eeb',
 u'da_DK': u'Chef, delte tjenester',
 u'de_DE': u'Shared Services Manager',
 u'en_US': u'Shared Services Manager',
 u'es_ES': u'Director de servicios compartidos',
 u'fi_FI': u'Jaettujen palvelujen p\xe4\xe4llikk\xf6',
 u'fr_FR': u'Responsable des services partag\xe9s',
 u'it_IT': u'Responsabile servizi condivisi',
 u'ja_JP': u'\u5171\u6709\u30b5\u30fc\u30d3\u30b9\u30de\u30cd\u30fc\u30b8\u30e3\u30fc',
 u'ko_KR': u'\uacf5\uc720 \uc11c\ube44\uc2a4 \uad00\ub9ac\uc790',
 u'nl_NL': u'Manager gedeelde services',
 u'no_NO': u'Leder for delte tjenester',
 u'pl_PL': u'Kierownik ds. us\u0142ug wsp\xf3lnych',
 u'pt_BR': u'Gerente de servi\xe7os compartilhados',
 u'ru_RU': u'\u041c\u0435\u043d\u0435\u0434\u0436\u0435\u0440 \u043f\u043e \u043e\u0431\u0449\u0435\u043a\u043e\u0440\u043f\u043e\u0440\u0430\u0442\u0438\u0432\u043d\u044b\u043c

In [310]:
myid = df3.iloc[1]['id']
myid

21391

In [326]:
len(keyword_dict)

19

In [152]:
keyword_keys = df3.iloc[1]['names'].keys()
keywords = df3.iloc[1]['names'].values()
keyword_keys

[u'ru_RU',
 u'ar_AE',
 u'fr_FR',
 u'pt_BR',
 u'zh_CN',
 u'fi_FI',
 u'da_DK',
 u'ko_KR',
 u'it_IT',
 u'sv_SE',
 u'no_NO',
 u'ja_JP',
 u'nl_NL',
 u'en_US',
 u'tr_TR',
 u'de_DE',
 u'pl_PL',
 u'cs_CZ',
 u'es_ES']

In [147]:
def build_search_keywords(keyword_dict):
    """
    Creates the list of keywords to be passed to the google ads api.  
    We need to append the translated word for "jobs" at the end of each of the keywords
    
    params:
        keyword_dict: dictionary of keywords, just a single keyword
    
    input: dictionary of languages and keywords, just a string keyword
    output: list of keywords with translated "job" appended to the end
    
    """
    if type(keyword_dict) == dict:
        keyword_keys = keyword_dict.keys();
        search_keywords = [];
        
        for i in keyword_keys:
            mykeyword = keyword_dict[i].encode('utf8').strip();
            job_trans = job_translations[str(i)];
            search_keywords.append(mykeyword + ' ' + job_trans);
        return search_keywords
    else:
        mykeyword = keyword_dict;
        mykeyword = mykeyword.encode('utf8').strip();
        job_trans = job_translations[country];
        search_keyword = mykeyword + ' ' + job_trans;
        
        return search_keyword

if __name__ == '__main__':
    keyword_dict = df3.iloc[1]['names'];
    search_keywords = build_search_keywords(keyword_dict);

In [143]:
type({}) == dict

True

In [145]:
search_keywords

['\xd0\x9c\xd0\xb5\xd0\xbd\xd0\xb5\xd0\xb4\xd0\xb6\xd0\xb5\xd1\x80-\xd0\xb4\xd0\xb8\xd0\xb5\xd1\x82\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xb3 jobs',
 '\xd9\x85\xd8\xaf\xd9\x8a\xd8\xb1 \xd8\xa3\xd8\xb8\xd9\x85\xd8\xa9 \xd8\xba\xd8\xb0\xd8\xa7\xd8\xa6\xd9\x8a\xd8\xa9 jobs',
 'Manager di\xc3\xa9t\xc3\xa9tiste emplois',
 'Gerente de nutri\xc3\xa7\xc3\xa3o jobs',
 '\xe8\x86\xb3\xe9\xa3\x9f\xe7\xbb\x8f\xe7\x90\x86 \xe8\x81\x8c\xe4\xbd\x8d',
 'Ravitsemusp\xc3\xa4\xc3\xa4llikk\xc3\xb6 jobs',
 'Kostchef jobs',
 '\xec\x8b\x9d\xec\x9d\xb4\xec\x9a\x94\xeb\xb2\x95 \xea\xb4\x80\xeb\xa6\xac\xec\x9e\x90 jobs',
 'Responsabile dietista offerte-di-lavoro',
 'Dietistansvarig jobs',
 'Leder for kosthold jobs',
 '\xe9\xa3\x9f\xe4\xba\x8b\xe7\x99\x82\xe6\xb3\x95\xe7\xae\xa1\xe7\x90\x86\xe8\x80\x85 jobs',
 'Dieetmanager jobs',
 'Dietary Manager jobs',
 'Beslenme Y\xc3\xb6neticisi jobs',
 'Ern\xc3\xa4hrungsmanager stellen',
 'Kierownik ds. dietetycznych jobs',
 'Mana\xc5\xbeer stravov\xc3\xa1n\xc3\xad jobs',
 'Director 

In [100]:
def add_to_dataframe(google_api_result, keyword_dict, myid, country, dataframe):
    """
    Adds the google api result and original keyword to the pandas dataframe
    
    input: 
    output: 
    """
    keywords = keyword_dict.values()
    
    for i in range(len(google_api_result)):
        mydata = [str(myid)] + google_api_result[i] + [country] + [keywords[i]]
        dataframe = dataframe.append(pd.DataFrame([tuple(mydata)], columns = pandas_schema))
    return dataframe

In [154]:
def add_to_dataframe2(google_api_result, keyword, myid, country, dataframe):
    """
    Adds the google api result and original keyword to the pandas dataframe
    
    input: 
    output: 
    """
    
    mydata = [str(myid)] + google_api_result + [country] + [keyword]
    dataframe = dataframe.append(pd.DataFrame([tuple(mydata)], columns = pandas_schema))
    return dataframe

In [250]:
"""
This retrieves keywords that are related to a given keyword.
The LoadFromStorage method is pulling credentials and properties from a
"googleads.yaml" file. By default, it looks for this file in your home
directory. For more information, see the "Caching authentication information"
section of our README.
"""

from googleads import adwords
PAGE_SIZE = 700

def main(client, mykeywords, country_code):
  # List for storing the results.
  mylist = []

  # Initialize appropriate service.
  targeting_idea_service = client.GetService(
      'TargetingIdeaService', version='v201609')

  # Construct selector object and retrieve related keywords.
  offset = 0
  selector = {
      'searchParameters': [
          {
              'xsi_type': 'RelatedToQuerySearchParameter',
              'queries': mykeywords
          },
          {
              # Network search parameter (optional)
              'xsi_type': 'NetworkSearchParameter',
              'networkSetting': {
                  'targetGoogleSearch': True,
                  'targetSearchNetwork': False,
                  'targetContentNetwork': False,
                  'targetPartnerSearchNetwork': False
              }
          },
          {
              'xsi_type': 'LocationSearchParameter',
              'locations': [{'id': country_code}]
              #'locations': [{'id': [2076, 2620] }]
              
          }
      ],
      'ideaType': 'KEYWORD',
      'requestType': 'STATS',#STATS or IDEAS
      'requestedAttributeTypes': ['KEYWORD_TEXT', 'SEARCH_VOLUME'], #['KEYWORD_TEXT', 'SEARCH_VOLUME','CATEGORY_PRODUCTS_AND_SERVICES'],
      'paging': {
          'startIndex': str(offset),
          'numberResults': str(PAGE_SIZE)
      }
  }

  more_pages = True
  while more_pages:
    page = targeting_idea_service.get(selector)

    # Display results.
    if 'entries' in page:
      for result in page['entries']:
        attributes = {}
        for attribute in result['data']:
          attributes[attribute['key']] = getattr(attribute['value'], 'value','0')
#         print ('Keyword with \'%s\' text and average monthly search volume '
#                '\'%s\' was found with Products and Services categories: %s.'
#                % (attributes['KEYWORD_TEXT'],
#                   attributes['SEARCH_VOLUME'],
#                   attributes['CATEGORY_PRODUCTS_AND_SERVICES']))
        
        mylist.append([str(attributes['KEYWORD_TEXT']), str(attributes['SEARCH_VOLUME'])])
        
    else:
      print 'No related keywords were found.'
    offset += PAGE_SIZE
    selector['paging']['startIndex'] = str(offset)
    more_pages = offset < int(page['totalNumEntries'])
    
    return mylist

In [229]:
country_mappings[country]

[2840, 2826, 2036, 2702, 2356, 2124]

In [356]:
if __name__ == '__main__':
    """Driver for google ads API requests, version 2
    
    This one sends the request 700 keywords at a time for one language/location . 
    This might be a better strategy because of the limits of the google ads API.
    
    Also has keyword deduplication built in
    """
    #initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage();

    #initialize pandas dataframe
    mydataframe = pd.DataFrame(columns = pandas_schema);

    #iterate over all the countries
    for i in job_translations.keys():
        country = i
        #country = 'es_ES'
        myids = []
        keywords = []
        request_keywords = []

        count = 0;

        num_iter = len(df3)

        for i in range(num_iter):
        #for i in range(1400):
        #for i in range(24500, len(df3)):
            try:
                myid        = df3.iloc[i]['id'];
                keyword     = df3.iloc[i]['names'][country];
                myids.append(myid);

                keywords.append(keyword);
                request_keywords.append(build_search_keywords(keyword)); #append the translated word for jobs
                
                count += 1;
            except:
                pass

            if count % 700 == 0:
                #first, remove duplicates
                keywords, myids, request_keywords = remove_duplicates(keywords, myids, request_keywords)

                if len(request_keywords) == 0:
                    pass
                else:
                    google_api_results = main(adwords_client, request_keywords, country_mappings[country]);
                    for i in range(len(google_api_results)):
                        api_result = google_api_results[i]
                        keyword = keywords[i]
                        myid = myids[i]
                        mydataframe = add_to_dataframe2(api_result, keyword, myid, country, mydataframe)

                    myids = []
                    keywords = []
                    request_keywords = []
                    time.sleep(2)

            if count == num_iter:
                google_api_results = main(adwords_client, request_keywords, country_mappings[country]);

                for i in range(len(google_api_results)):
                    api_result = google_api_results[i]
                    keyword = keywords[i]
                    myid = myids[i]

                    mydataframe = add_to_dataframe2(api_result, keyword, myid, country, mydataframe)

In [359]:
class RemoveDuplicates(object):
    def __init__(self):
        pass
    
    @staticmethod
    def list_duplicates(seq):
      seen = set()
      seen_add = seen.add
      # adds all elements it doesn't know yet to seen and all other to seen_twice
      seen_twice = set( x for x in seq if x in seen or seen_add(x) )
      # turn the set into a list (as requested)
      return list(seen_twice)

    @staticmethod
    def remove_duplicates(keywords, ids, request_keywords):
        mydups = RemoveDuplicates.list_duplicates(keywords)

        #dictionary to keep track of times duplicate are seen.
        dup_count = {}
        for i in mydups:
            dup_count[i] = 0

        for i in range(len(keywords)):
            keyword = keywords[i]
            if keyword in mydups:
                dup_count[keyword] += 1
                if dup_count[keyword] > 1:
                    keywords[i] = 'remove'
                    ids[i] = 'remove'
                    request_keywords[i] = 'remove'

        while 'remove' in keywords: 
            keywords.remove('remove')
        while 'remove' in ids:
            ids.remove('remove')
        while 'remove' in request_keywords:
            request_keywords.remove('remove')

        return keywords, ids, request_keywords

In [358]:
list_duplicates(keywords)

[u'Encargado de facturaci\xf3n',
 u'Ingeniero civil',
 u'Consultor de energ\xeda solar',
 u'Instructor de equitaci\xf3n',
 u'Instructor de tiro con armas de fuego',
 u'Oficial de centro de detenci\xf3n',
 u'Instructor de aquagym',
 u'Instructor de actividades al aire libre',
 u'Instructor de gimnasia']

In [348]:
keywords

[]

In [322]:
remove_duplicates(keywords)

In [302]:
len(request_keywords)
request_keywords

['Consultor de tecnolog\xc3\xada SAP empleos',
 'Director de nutrici\xc3\xb3n empleos',
 'Director de servicios compartidos empleos',
 'Vicepresidente de desarrollo de operaciones empresariales empleos',
 'Director de RF empleos',
 'Comandante empleos',
 'Director de proyectos de hardware empleos',
 'Especialista cardiovascular empleos',
 'Jefe de ejecuci\xc3\xb3n de operaciones empleos',
 'Mentor empleos',
 'Coordinador de prestaci\xc3\xb3n de servicios empleos',
 'Director general de inversiones empleos',
 'Gerente de especialidad empleos',
 'Especialista en desarrollo empleos',
 'Jefe profesional de tienda empleos',
 'Jefe de fletes empleos',
 'Ingeniero de puesta en servicio empleos',
 'Administrador de datos empleos',
 'Supervisor cl\xc3\xadnico empleos',
 'Ejecutivo de asistencia al cliente empleos',
 'Gerente general de finanzas empleos',
 'Arquitecto t\xc3\xa9cnico jefe empleos',
 'Gerente de soporte t\xc3\xa9cnico de TI empleos',
 'Jefe de fiscalidad empleos',
 'Director de ha

In [289]:
len(keywords)

199

In [354]:
mydataframe

,id,search_keyword,volume,location,keyword
0,12005,consultor de tecnología sap empleos,0,es_ES,Consultor de tecnología SAP
0,21391,director de nutrición empleos,0,es_ES,Director de nutrición
0,6914,director de servicios compartidos empleos,0,es_ES,Director de servicios compartidos
0,10484,vicepresidente de desarrollo de operaciones em...,0,es_ES,Vicepresidente de desarrollo de operaciones em...
0,5139,director de rf empleos,0,es_ES,Director de RF
0,8904,comandante empleos,0,es_ES,Comandante
0,21327,director de proyectos de hardware empleos,0,es_ES,Director de proyectos de hardware
0,11928,especialista cardiovascular empleos,0,es_ES,Especialista cardiovascular
0,13983,jefe de ejecución de operaciones empleos,0,es_ES,Jefe de ejecución de operaciones
0,593,mentor empleos,0,es_ES,Mentor


In [203]:
keyword_list

NameError: name 'keyword_list' is not defined

In [191]:
mydataframe

,id,search_keyword,volume,location,keyword


In [355]:
mydataframe = mydataframe.dropna();
write_dataframe(client, '/user/luzhang/career_page_seo/google/volume/results.avro', mydataframe, overwrite = True, schema = schema);

In [130]:
a = 3

In [133]:
a += 1

In [134]:
a

4

In [136]:
1300 % 700

600

In [137]:
0 % 700

0

In [ ]:
raise Exception('ugh')

In [ ]:
if __name__ == '__main__':
    """Driver for google ads API requests, version 1"""
    
    #initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage();

    #initialize pandas dataframe
    mydataframe = pd.DataFrame(columns = pandas_schema);

    #for i in range(len(df3)):
    for i in range(2):
        
        keyword_dict      = df3.iloc[i]['names'];
        myid              = df3.iloc[i]['id']
        search_keywords   = build_search_keywords(keyword_dict);
        google_api_result = []
        
        
        country_codes = country_mappings[country]
        
        for i in search_keywords:
            google_api_result += main(adwords_client, [i], country_codes)
        
        mydataframe = add_to_dataframe(google_api_result, keyword_dict, myid, country, mydataframe)
        time.sleep(2)